# Creating code for post processing the segmented sheets
* Just gave up on second step segmentation
* might be good to remove holes and modified Frangi filter that enhances surfaceness rather than vesselness

### Create the object to run inferences

In [2]:
case_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_images/2290837.tif"

In [3]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
import os
import json

config_content = {
  "architecture": "STU-Net",
  "patch_size": [128, 128, 128],
  "device": "cpu",
  "batch_size": 1,
  "num_workers": 4,

  "infer_sw_batch_size": 1,
  "blend_mode": "gaussian",
  "deep_supervision": False,
  "activation": True,

  "infer_overlap": 0.5,
  "TTA": True,
  "TH": 0.5,

  "checkpoint_path": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/Focal_Tversky_CLDICE_DAsafe/model_epoch_40.pth", 
  "dataset_path_imgs": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images",
  "pred_save_dir": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash"
}


# 🧪 Initialize Inference Object
infer_object = VesuviusInferer(config_content)

print(f"🚀 Starting inference on dataset: {config_content['dataset_path_imgs']}")
print("📋 Using the following configuration:")
for key, value in config_content.items():
    print(f"  🔹 {key}: {value}")


### Select manually a case to run!
input_data = {
    'image': str(case_path),
    'gt': None
}
logits_pred, pred = infer_object.infer(
    input=input_data, 
    test=True, 
    threshold=0.5 # CHANGED FROM 0.5
)
infer_object.save_nifti(
    torch_tensor=pred, 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.nii.gz", 
    real_file=None
)

infer_object.save_tiff(
    torch_tensor=pred, 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.tif"
)

Loading weights from: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/Focal_Tversky_CLDICE_DAsafe/model_epoch_40.pth
🚀 Starting inference on dataset: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
📋 Using the following configuration:
  🔹 architecture: STU-Net
  🔹 patch_size: [128, 128, 128]
  🔹 device: cpu
  🔹 batch_size: 1
  🔹 num_workers: 4
  🔹 infer_sw_batch_size: 1
  🔹 blend_mode: gaussian
  🔹 deep_supervision: False
  🔹 activation: True
  🔹 infer_overlap: 0.5
  🔹 TTA: True
  🔹 TH: 0.5
  🔹 checkpoint_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/Focal_Tversky_CLDICE_DAsafe/model_epoch_40.pth
  🔹 dataset_path_imgs: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
  🔹 pred_save_dir: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


  0%|          | 0/64 [00:00<?, ?it/s]Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
100%|██████████| 64/64 [17:42<00:00, 16.60s/it]


Doing axes: (3,)


100%|██████████| 64/64 [17:58<00:00, 16.84s/it]


Doing axes: (4,)


100%|██████████| 64/64 [17:50<00:00, 16.73s/it]


Doing axes: (3, 4)


100%|██████████| 64/64 [17:49<00:00, 16.71s/it]


### Create object for testing

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))
import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import topometrics.leaderboard

def score_single_tif(
    gt_path,
    pred_path,
    surface_tolerance,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
):
    gt: np.ndarray = load_volume(gt_path)
    pr: np.ndarray = load_volume(pred_path)

    #install_dependencies()
    # The import is here to ensure dependencies are loaded first.
    try:
        # Use a standard import now that the path is reliably set.
        import topometrics.leaderboard
    except Exception as err:
        raise HostVisibleError(f'Failed to import topometrics after installation: {err}')

    score_report = topometrics.leaderboard.compute_leaderboard_score(
        predictions=pr,
        labels=gt,
        dims=(0, 1, 2),
        spacing=(1.0, 1.0, 1.0),  # (z, y, x)
        surface_tolerance=surface_tolerance,  # in spacing units
        voi_connectivity=voi_connectivity,
        voi_transform=voi_transform,
        voi_alpha=voi_alpha,
        combine_weights=(topo_weight, surface_dice_weight, voi_weight),  # (Topo, SurfaceDice, VOI)
        fg_threshold=None,  # None => legacy "!= 0"; else uses "x > threshold"
        ignore_label=2,  # voxels with this GT label are ignored
        ignore_mask=None,  # or pass an explicit boolean mask
    )
    
    return {
        'image_score': np.clip(score_report.score, 0.0, 1.0),
        'topo_score': score_report.topo.toposcore,
        'surface_dice': score_report.surface_dice,
        'voi_score': score_report.voi.voi_score,
        'voi_split': score_report.voi.voi_split,
        'voi_merge': score_report.voi.voi_merge
    }

def load_volume(path):
    im = Image.open(path)
    slices = []
    for i, page in enumerate(ImageSequence.Iterator(im)):
        slice_array = np.array(page)
        slices.append(slice_array)
    volume = np.stack(slices, axis=0)
    return volume


cases_list = []
for i in range(20):
    here_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter{i+1}_2290837_th06.tif"
    cases_list.append(here_path)
for pred_path in cases_list:
    score_report = score_single_tif(
        gt_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/test_labels/2290837.tif",
        pred_path=pred_path,
        surface_tolerance=2.0,
        voi_connectivity=26,
        voi_transform='one_over_one_plus',
        voi_alpha=0.3,
        topo_weight=0.3,
        surface_dice_weight=0.35,
        voi_weight=0.35,
    )
    print(pred_path)
    print(score_report)

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter1_2290837_th06.tif
{'image_score': np.float64(0.45228928401352025), 'topo_score': 0.0833147694364001, 'surface_dice': 0.6708227554855642, 'voi_score': 0.5500196821790077, 'voi_split': 1.4193430016698145, 'voi_merge': 1.3077128498487447}


In [ ]:
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter1_2290837_th05.tif
{'image_score': np.float64(0.45768911520769795), 'topo_score': 0.08993009635367467, 'surface_dice': 0.6805804373994035, 'voi_score': 0.5500198091765837, 'voi_split': 1.4793721181518924, 'voi_merge': 1.247682334044819}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter2_2290837_th05.tif
{'image_score': np.float64(0.4573842701064049), 'topo_score': 0.08917754324611253, 'surface_dice': 0.6803342540241168, 'voi_score': 0.5500400520689436, 'voi_split': 1.4783647125598003, 'voi_merge': 1.2484667017378692}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter3_2290837_th05.tif
{'image_score': np.float64(0.45712028771369806), 'topo_score': 0.08917754324611253, 'surface_dice': 0.6794846494874568, 'voi_score': 0.5501354211978697, 'voi_split': 1.4763362010799943, 'voi_merge': 1.2494446489186675}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter4_2290837_th05.tif
{'image_score': np.float64(0.4559664602409208), 'topo_score': 0.08688077124107091, 'surface_dice': 0.6782316408386639, 'voi_score': 0.5500604416430488, 'voi_split': 1.4705045826125571, 'voi_merge': 1.2561021942291948}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter5_2290837_th05.tif
{'image_score': np.float64(0.44300796896669337), 'topo_score': 0.047834597504281866, 'surface_dice': 0.6749676240087409, 'voi_score': 0.5497683466067129, 'voi_split': 1.4665449239626627, 'voi_merge': 1.2632815333819787}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter6_2290837_th05.tif
{'image_score': np.float64(0.4359294359501279), 'topo_score': 0.03368439189334712, 'surface_dice': 0.6672334410165159, 'voi_score': 0.5494068972181233, 'voi_split': 1.463909288757047, 'voi_merge': 1.2699060617200937}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter7_2290837_th05.tif
{'image_score': np.float64(0.42833716232349267), 'topo_score': 0.02065777308149518, 'surface_dice': 0.6594670404579974, 'voi_score': 0.5466467606821285, 'voi_split': 1.4735367891751, 'voi_merge': 1.2909128940301853}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter8_2290837_th05.tif
{'image_score': np.float64(0.4221530667048113), 'topo_score': 0.014121768652201785, 'surface_dice': 0.6482468880619957, 'voi_score': 0.5458003579641492, 'voi_split': 1.4504646219429345, 'voi_merge': 1.3234412297011136}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter9_2290837_th05.tif
{'image_score': np.float64(0.41413063068888334), 'topo_score': 0.012693178194488734, 'surface_dice': 0.631342442586901, 'voi_score': 0.541008063786061, 'voi_split': 1.4649225358216145, 'voi_merge': 1.3630817456786155}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter10_2290837_th05.tif
{'image_score': np.float64(0.4019216575268936), 'topo_score': 0.017884147544217244, 'surface_dice': 0.6103401563851356, 'voi_score': 0.5226781672252313, 'voi_split': 1.6364961725830431, 'voi_merge': 1.4075811883721816}

#Filled_th1000_iter1_2290837_th06.tif
{'image_score': np.float64(0.4540724610951989), 'topo_score': 0.08895760242611644, 'surface_dice': 0.6708014882691639, 'voi_score': 0.5502990270661616, 'voi_split': 1.4150610214337869, 'voi_merge': 1.3089184322543475}
#Filled_th1000_iter2_2290837_th06.tif
{'image_score': np.float64(0.4541016265778933), 'topo_score': 0.08967212132110532, 'surface_dice': 0.670253579162178, 'voi_score': 0.5503178213565697, 'voi_split': 1.4136640458670724, 'voi_merge': 1.3101085402779913}
#Filled_th1000_iter3_2290837_th06.tif
{'image_score': np.float64(0.4528220461413176), 'topo_score': 0.08688077124107091, 'surface_dice': 0.6689408027268444, 'voi_score': 0.5503672394702879, 'voi_split': 1.4116056391239973, 'voi_merge': 1.3116230724448479}
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter4_2290837_th06.tif
{'image_score': np.float64(0.45060664687088914), 'topo_score': 0.0817888579082609, 'surface_dice': 0.6670616537765407, 'voi_score': 0.550281173361776, 'voi_split': 1.4061567635492933, 'voi_merge': 1.3180192177998002}
#Filled_th1000_iter5_2290837_th06.tif
{'image_score': np.float64(0.43851480053834013), 'topo_score': 0.04584878482331669, 'surface_dice': 0.6636433983773303, 'voi_score': 0.5499570733122271, 'voi_split': 1.4026558733647054, 'voi_merge': 1.3250899121823918}
#Filled_th1000_iter6_2290837_th06.tif
{'image_score': np.float64(0.4328934648677397), 'topo_score': 0.036416979067222396, 'surface_dice': 0.656066151780207, 'voi_score': 0.5495577657842873, 'voi_split': 1.4029905006846255, 'voi_merge': 1.3291592523255713}
#Filled_th1000_iter7_2290837_th06.tif
{'image_score': np.float64(0.42504746056984055), 'topo_score': 0.02108697703892462, 'surface_dice': 0.6493561358487372, 'voi_score': 0.5469906283174432, 'voi_split': 1.414131369302559, 'voi_merge': 1.346484920572005}
#Filled_th1000_iter8_2290837_th06.tif
{'image_score': np.float64(0.41968649976127137), 'topo_score': 0.015654375935233857, 'surface_dice': 0.6396213708302455, 'voi_score': 0.5460648776860435, 'voi_split': 1.3945760027726886, 'voi_merge': 1.376371436278339}
#Filled_th1000_iter9_2290837_th06.tif
{'image_score': np.float64(0.41156282293468827), 'topo_score': 0.013238157040882544, 'surface_dice': 0.6234328445904739, 'voi_score': 0.5411139434735931, 'voi_split': 1.4139184373448157, 'voi_merge': 1.412880256110037}
#Filled_th1000_iter10_2290837_th06.tif
{'image_score': np.float64(0.3999980641827964), 'topo_score': 0.01866529421856236, 'surface_dice': 0.6033449470409616, 'voi_score': 0.5235078412939744, 'voi_split': 1.579960161915782, 'voi_merge': 1.4540100490382166}

# bigger1000_2290837_th05_B
{'image_score': np.float64(0.4571586251860389), 'topo_score': 0.0880720365116566, 'surface_dice': 0.6806239621724534, 'voi_score': 0.550053221349095, 'voi_split': 1.4794637417589183, 'voi_merge': 1.2472225811108175}
# bigger1000_2290837_th06_B
{'image_score': np.float64(0.4525479134612837), 'topo_score': 0.08362680977136788, 'surface_dice': 0.6709918890905741, 'voi_score': 0.550322026709064, 'voi_split': 1.4155212803140023, 'voi_merge': 1.3082050197217456}
# bigger10000_2290837_th05_B (IMPROVED A BIT)
{'image_score': np.float64(0.4609813939137337), 'topo_score': 0.09472636815920399, 'surface_dice': 0.6832015861242192, 'voi_score': 0.5526940809214164, 'voi_split': 1.4451191639910366, 'voi_merge': 1.2526114282675465}
# bigger10000_2290837_th06_B
{'image_score': np.float64(0.45751619846359504), 'topo_score': 0.0950142902508733, 'surface_dice': 0.6736287622165399, 'voi_score': 0.5521195560358401, 'voi_split': 1.3919077958972967, 'voi_merge': 1.3120986043341683}

# th 0.6 basic
{'image_score': np.float64(0.4471887115585188),
 'topo_score': 0.0658653634482455,
 'surface_dice': 0.6716796123490321,
 'voi_score': 0.5495463948625254,
 'voi_split': 1.4249374889394884,
 'voi_merge': 1.307337767821196}

# th 0.5 basic 
{'image_score': np.float64(0.4528044081004991),
 'topo_score': 0.0745224924329402,
 'surface_dice': 0.6802427208108751,
 'voi_score': 0.5496077373908878,
 'voi_split': 1.4848847490028618,
 'voi_merge': 1.246713516257051}

### Select each individual object, and remove bellow threshold

In [6]:
import tifffile
import numpy as np
from skimage.measure import label
import nibabel as nib
import numpy as np

def split_by_object(input_path, output_path):
    print("Loading image...")
    volume = tifffile.imread(input_path)

    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = volume > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    print("Labeling individual elements...")
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    print(f"Initial count: {num_features} elements.")
    print(f"Found {num_features} individual elements.")
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)

    # 4. Save the new TIFF
    print(f"Saving to {output_path}")
    tifffile.imwrite(output_path, final_volume)

    nii_data = np.transpose(binary_mask, (2, 1, 0)).astype(np.uint16)
    nii_data = np.flip(nii_data, axis=0) 
    nii_data = np.flip(nii_data, axis=1)
    output_path_nii = output_path.replace('.tif', '_B.nii.gz')
    nib.save(nib.Nifti1Image(nii_data.astype(np.uint16), np.eye(4)), output_path_nii)

    print("Done!")
    return final_volume

input_path = '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif'
output_path = '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/categorical_2290837_th06.tif'

final_volume = split_by_object(input_path, output_path)

Loading image...
Labeling individual elements...
Initial count: 79 elements.
Found 79 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/categorical_2290837_th06.tif
Done!


In [10]:
def split_by_object_np(binary_mask):
    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = binary_mask > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    print("Labeling individual elements...")
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    print(f"Found {num_features} individual elements.")
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)
    return final_volume

In [14]:
import tifffile
import numpy as np
from skimage.morphology import remove_small_objects
from skimage.measure import label
import nibabel as nib
import numpy as np

def remove_bellow_th(input_path, th, output_path):
    binary_mask = tifffile.imread(input_path)
    labeled_volume = split_by_object_np(binary_mask) 
    
    # 3. Remove Small Objects
    # connectivity=1 ensures we stick to the same face-to-face logic as before
    print(f"Removing objects smaller than {th} voxels...")
    filtered_volume = remove_small_objects(labeled_volume, min_size=th, connectivity=1)

    # Note: remove_small_objects might return a boolean if the input was boolean. 
    # We ensure it stays labeled by multiplying by the original labels (masking).
    if filtered_volume.dtype == bool:
        filtered_volume =  label(filtered_volume)
    num_features = np.max(filtered_volume)
    print(f"After filtering, count: {num_features} elements.")
    if output_path!=None:
        # 4. Save
        print(f"Saving to {output_path}")
        tifffile.imwrite(output_path, filtered_volume.astype(np.uint16))

        # Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
        output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
        nii_data = np.transpose(filtered_volume, (2, 1, 0)).astype(np.uint16)
        nii_data = np.flip(nii_data, axis=0) 
        nii_data = np.flip(nii_data, axis=1)
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
        nii_data[nii_data>0.5] = 1 
        output_path_nii = f"{output_path.replace('.tif', '_B.nii.gz')}"
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
        filtered_volume_b = np.zeros_like(filtered_volume)
        filtered_volume_b[filtered_volume>0.5] = 1
        output_path = f"{output_path.replace('.tif', '_B.tif')}"
        tifffile.imwrite(f"{output_path}", filtered_volume_b.astype(np.uint16))
    print("Done.")
    return filtered_volume
    

final_volume_th = remove_bellow_th(
    input_path='/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.tif', 
    th=1000, 
    output_path='/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/bigger1000_2290837_th05.tif')

Labeling individual elements...
Found 83 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/bigger1000_2290837_th05.tif
Done.


In [ ]:
import tifffile
import numpy as np
from skimage.morphology import remove_small_objects
from skimage.measure import label

def remove_outside_th(labeled_volume, th_down, th_up, output_path):
    # 3. Remove Small Objects
    # connectivity=1 ensures we stick to the same face-to-face logic as before
    print(f"Removing objects bigger than {th_up} and smaller than {th_down} voxels...")
    # volume with bigger structures
    filtered_volume = remove_small_objects(labeled_volume, min_size=th_up, connectivity=1)
    # remaining (smaller structures)
    filtered_volume = labeled_volume - filtered_volume
    # remove very small
    filtered_volume = remove_small_objects(filtered_volume, min_size=th_down, connectivity=1)
    # Note: remove_small_objects might return a boolean if the input was boolean. 
    # We ensure it stays labeled by multiplying by the original labels (masking).
    if filtered_volume.dtype == bool:
        filtered_volume =  label(filtered_volume)
    num_features = np.max(filtered_volume)
    print(f"After filtering, count: {num_features} elements.")
        
    # 4. Save
    print(f"Saving to {output_path}")
    tifffile.imwrite(output_path, filtered_volume.astype(np.uint16))

    filtered_volume[filtered_volume>0.5] = 1
    output_path = f"{output_path.replace('.tif', '_B.tif')}"
    tifffile.imwrite(f"{output_path}", filtered_volume.astype(np.uint16))

    print("Done.")

remove_outside_th(
    labeled_volume=final_volume, 
    th_down=10000,
    th_up=100000, 
    output_path='/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/filter_inside__categorical_2290837.tif')

In [ ]:
# OLD

def split_all_segments(labeled_volume, iterations=1):
    print(f"Eroding all segments simultaneously (Iterations: {iterations})...")
    print(f"Then tries to separate them...")
    # Work on a copy so we don't modify the original in memory
    eroded_volume = labeled_volume.copy()
    
    # ERODING!
    for i in range(iterations):
        print(f"  - Pass {i+1}/{iterations}...")
        
        # find_boundaries(mode='inner') finds pixels that belong to an object 
        # but touch a different value (either background 0 OR a neighbor object).
        # This is incredibly fast compared to looping.
        boundaries = find_boundaries(eroded_volume, mode='inner')
        
        # Set the boundaries to 0 (background)
        eroded_volume[boundaries] = 0
    
    # SPLITING BY INDIVIDUAL OBJECT 
    splited_labels = split_by_object_np(eroded_volume)
    unique_ids = np.unique(splited_labels)
    num_elements = len(unique_ids) - (1 if 0 in unique_ids else 0)

    # REMOVE SMALL OBJECTS
    filtered_volume = remove_bellow_th(splited_labels, th=1000, output_path=None)
    print(f"Total individual segments found: {num_elements}")

    return filtered_volume

#### Mine fill holes

In [28]:
import numpy as np
import tifffile
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm

# --- NEW IMPORTS REQUIRED ---
from skimage.morphology import remove_small_objects
from skimage.measure import label

def remove_bellow_th_np(labeled_volume, th, output_path):
    # 3. Remove Small Objects
    print(f"Removing objects smaller than {th} voxels...")
    
    # remove_small_objects expects an ARRAY, not a list of slices
    filtered_volume = remove_small_objects(labeled_volume, min_size=th, connectivity=1)

    # Ensure it stays labeled if it somehow converts to boolean
    if filtered_volume.dtype == bool:
        filtered_volume = label(filtered_volume)
        
    num_features = np.max(filtered_volume)
    print(f"After filtering, count: {num_features} elements.")
    
    if output_path is not None:
        print(f"Saving to {output_path}")
        tifffile.imwrite(output_path, filtered_volume.astype(np.uint16))

        # Save as .nii.gz 
        output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
        nii_data = np.transpose(filtered_volume, (2, 1, 0)).astype(np.uint16)
        nii_data = np.flip(nii_data, axis=0) 
        nii_data = np.flip(nii_data, axis=1)
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
        
        # Save Binary version
        nii_data[nii_data > 0.5] = 1 
        output_path_nii_b = f"{output_path.replace('.tif', '_B.nii.gz')}"
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii_b)
        
        filtered_volume_b = np.zeros_like(filtered_volume)
        filtered_volume_b[filtered_volume > 0.5] = 1
        output_path_b = f"{output_path.replace('.tif', '_B.tif')}"
        tifffile.imwrite(output_path_b, filtered_volume_b.astype(np.uint16))
        
    print("Done.")
    return filtered_volume

def dilate_labeled_volume_3d(labeled_array, iterations=1):
    print(f"Starting 3D dilation ({iterations} iterations)...")
    dilated_volume = np.zeros_like(labeled_array)
    struct = ndimage.generate_binary_structure(3, 1) 
    object_slices = ndimage.find_objects(labeled_array)
    
    for i, slc in enumerate(tqdm(object_slices, desc="Dilating objects")):
        if slc is None:
            continue
        label_id = i + 1
        sub_vol = labeled_array[slc] == label_id
        dilated_sub_vol = ndimage.binary_dilation(sub_vol, structure=struct, iterations=iterations)
        
        # Prevent overwriting existing labels
        mask = (dilated_volume[slc] == 0) & dilated_sub_vol
        dilated_volume[slc][mask] = label_id
        
    return dilated_volume

def erode_labeled_volume_3d(labeled_array, iterations=1):
    print(f"Starting 3D erosion ({iterations} iterations)...")
    eroded_volume = np.zeros_like(labeled_array)
    struct = ndimage.generate_binary_structure(3, 1) 
    object_slices = ndimage.find_objects(labeled_array)
    
    for i, slc in enumerate(tqdm(object_slices, desc="Eroding objects")):
        if slc is None:
            continue
        label_id = i + 1
        sub_vol = labeled_array[slc] == label_id
        eroded_sub_vol = ndimage.binary_erosion(sub_vol, structure=struct, iterations=iterations)
        eroded_volume[slc][eroded_sub_vol] = label_id
        
    return eroded_volume

def whole_process(input_path, iterations=3, filter_th=1000, output_path=None):
    print(f"Reading {input_path}...")
    volume = tifffile.imread(input_path)

    # 1. Prepare Binary Mask
    binary_mask = volume > 0

    # 2. Label
    print("Labeling individual elements...")
    labeled_array = label(binary_mask, connectivity=None)
    
    # --- ERROR WAS HERE ---
    # Don't pass 'ndimage.find_objects' (a list) to the removal function.
    # Pass the 'labeled_array' (the image volume) directly.
    
    # 3. Remove Small Objects
    filtered_volume = remove_bellow_th_np(labeled_array, th=filter_th, output_path=None)

    # 4. Dilation (Fill holes start)
    dilated_volume = dilate_labeled_volume_3d(filtered_volume, iterations=iterations)

    # 5. Erosion (Fill holes end - closing operation)
    new_eroded_volume = erode_labeled_volume_3d(dilated_volume, iterations=iterations)

    # 6. Count Stats
    unique_ids = np.unique(new_eroded_volume)
    # Subtract 1 for background (0) if present
    num_elements = len(unique_ids) - (1 if 0 in unique_ids else 0)
    print(f"Total individual segments found: {num_elements}")

    if output_path is not None:
        print(f"Saving to {output_path}...")
        # Fixed typo: 'eroded_volume' -> 'new_eroded_volume'
        tifffile.imwrite(output_path, new_eroded_volume.astype(np.uint16))

    return new_eroded_volume

# --- RUN LOOP ---
base_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/"
input_file = f"{base_path}2290837_th05.tif"

for i in range(10):
    output_file = f"{base_path}Filled_th1000_iter{i+1}_2290837_th05.tif"
    eroded_volume = whole_process(
        input_file, 
        iterations=i+1, 
        filter_th=1000,
        output_path=output_file
    )

Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.tif...
Labeling individual elements...
Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Starting 3D dilation (1 iterations)...


Dilating objects: 100%|██████████| 76/76 [00:01<00:00, 52.27it/s]


Starting 3D erosion (1 iterations)...


Eroding objects: 100%|██████████| 76/76 [00:00<00:00, 118.33it/s]


Total individual segments found: 16
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter1_2290837_th05.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.tif...
Labeling individual elements...
Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Starting 3D dilation (2 iterations)...


...

Total individual segments found: 14
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter9_2290837_th05.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th05.tif...
Labeling individual elements...
Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Starting 3D dilation (10 iterations)...


Dilating objects: 100%|██████████| 76/76 [00:01<00:00, 46.87it/s]


Starting 3D erosion (10 iterations)...


Eroding objects: 100%|██████████| 76/76 [00:00<00:00, 80.55it/s]


Total individual segments found: 14
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/Filled_th1000_iter10_2290837_th05.tif...


#### Gemini new fill holes

In [32]:
import numpy as np
import tifffile
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.measure import label

def fill_internal_holes(labeled_array, hole_size_th):
    """
    Fills holes within labeled objects without changing their outer edges.
    Args:
        labeled_array: The 3D labeled volume.
        hole_size_th: Maximum size (in voxels) of a hole to fill.
    """
    print(f"Filling holes smaller than {hole_size_th} voxels (per object)...")
    
    # We create a new array to store the result
    filled_volume = np.zeros_like(labeled_array)
    
    # find_objects returns a list of slices (bounding boxes) for each label
    object_slices = ndimage.find_objects(labeled_array)
    
    for i, slc in enumerate(tqdm(object_slices, desc="Processing objects")):
        if slc is None:
            continue
            
        label_id = i + 1
        
        # 1. Extract the single object within its bounding box
        # We perform the operation on a smaller sub-volume for speed
        sub_vol_mask = (labeled_array[slc] == label_id)
        
        # 2. Fill holes
        # This function finds background (0) regions enclosed by foreground (1).
        # If the background region is smaller than 'area_threshold', it becomes 1.
        # It DOES NOT expand the outer edge.
        filled_sub_vol = remove_small_holes(
            sub_vol_mask, 
            area_threshold=hole_size_th,
            connectivity=1 # 1 = 6-connected neighbors in 3D
        )
        
        # 3. Place back into the volume
        # We use the mask to ensure we don't overwrite other overlapping labels
        # (though ideally, labels shouldn't overlap in a labeled_array)
        current_region = filled_volume[slc]
        
        # Where the filled sub-volume is True, set the label_id
        current_region[filled_sub_vol] = label_id
        
        filled_volume[slc] = current_region

    return filled_volume

def remove_bellow_th_np(labeled_volume, th, output_path):
    print(f"Removing objects smaller than {th} voxels...")
    
    # Remove small foreground objects (dust)
    filtered_volume = remove_small_objects(labeled_volume, min_size=th, connectivity=1)

    if filtered_volume.dtype == bool:
        filtered_volume = label(filtered_volume)
        
    num_features = np.max(filtered_volume)
    print(f"After filtering, count: {num_features} elements.")
    
    if output_path is not None:
        print(f"Saving to {output_path}")
        tifffile.imwrite(output_path, filtered_volume.astype(np.uint16))
        
        # Save .nii.gz for visualization
        output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
        nii_data = np.transpose(filtered_volume, (2, 1, 0)).astype(np.uint16)
        # Flip correction (optional, depends on your specific data orientation)
        nii_data = np.flip(nii_data, axis=0) 
        nii_data = np.flip(nii_data, axis=1)
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

    print("Done.")
    return filtered_volume

def whole_process(input_path, hole_size_th=5000, object_size_th=1000, output_path=None):
    """
    object_size_th: remove specks smaller than this (dust)
    hole_size_th: fill internal voids smaller than this (bubbles)
    """
    print(f"Reading {input_path}...")
    volume = tifffile.imread(input_path)

    # 1. Prepare Binary Mask
    binary_mask = volume > 0

    # 2. Label
    print("Labeling individual elements...")
    labeled_array = label(binary_mask, connectivity=None)

    # 3. Fill Internal Holes (Topological Repair)
    # We do this BEFORE removing small objects, in case a "small object" 
    # is just a fragmented part of a big sheet that needs filling.
    # Note: If your holes are massive, increase hole_size_th.
    filled_volume = fill_internal_holes(labeled_array, hole_size_th=hole_size_th)

    # 4. Remove Small Objects (Dust Cleaning)
    final_volume = remove_bellow_th_np(filled_volume, th=object_size_th, output_path=None)

    # 5. Count Stats
    unique_ids = np.unique(final_volume)
    num_elements = len(unique_ids) - (1 if 0 in unique_ids else 0)
    print(f"Total individual segments found: {num_elements}")

    if output_path is not None:
        print(f"Saving to {output_path}...")
        tifffile.imwrite(output_path, final_volume.astype(np.uint16))

    return final_volume
# --- RUN LOOP ---
base_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/"
input_file = f"{base_path}2290837_th06.tif"

for i in range(20):
    output_file = f"{base_path}GEMINI_Filled_th1000_iter{i+1}_2290837_th06.tif"
    eroded_volume = whole_process(
        input_file, 
        hole_size_th=1000*(i+1), 
        object_size_th=1000, 
        output_path=output_file
    )

Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif...
Labeling individual elements...
Filling holes smaller than 1000 voxels (per object)...


Processing objects: 100%|██████████| 79/79 [00:02<00:00, 38.21it/s]


Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Total individual segments found: 17
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter1_2290837_th06.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif...
Labeling individual elements...
Filling holes smaller than 2000 voxels (per object)...


Processing objects: 100%|██████████| 79/79 [00:02<00:00, 39.47it/s]


Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Total individual segments found: 18
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter2_2290837_th06.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif...
Labeling individual elements...
Filling holes smaller than 3000 voxels (per object)...


...

Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Total individual segments found: 21
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter18_2290837_th06.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif...
Labeling individual elements...
Filling holes smaller than 19000 voxels (per object)...


Processing objects: 100%|██████████| 79/79 [00:02<00:00, 38.20it/s]


Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Total individual segments found: 21
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter19_2290837_th06.tif...
Reading /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif...
Labeling individual elements...
Filling holes smaller than 20000 voxels (per object)...


Processing objects: 100%|██████████| 79/79 [00:02<00:00, 38.65it/s]


Removing objects smaller than 1000 voxels...
After filtering, count: 76 elements.
Done.
Total individual segments found: 21
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/GEMINI_Filled_th1000_iter20_2290837_th06.tif...


In [20]:
import numpy as np
import tifffile
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm

import numpy as np
import tifffile
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm

def dilate_labeled_volume_3d(labeled_array, iterations=1):
    print(f"Starting 3D dilation ({iterations} iterations)...")
    
    # Create output array
    dilated_volume = np.zeros_like(labeled_array)
    
    # 3D structure (6-connectivity: faces touch)
    struct = ndimage.generate_binary_structure(3, 1) 
    
    # Find bounding boxes for efficiency
    object_slices = ndimage.find_objects(labeled_array)
    
    for i, slc in enumerate(tqdm(object_slices, desc="Dilating objects")):
        if slc is None:
            continue
            
        label_id = i + 1
        
        # 1. Extract sub-volume
        sub_vol = labeled_array[slc] == label_id
        
        # 2. Perform 3D dilation
        # Note: We use brute_force=True for very thin structures to ensure stability
        dilated_sub_vol = ndimage.binary_dilation(sub_vol, structure=struct, iterations=iterations)
        
        # 3. Place back into volume
        # Caution: If objects are very close, dilation might cause them to overlap.
        # This logic prevents overwriting existing labels.
        mask = (dilated_volume[slc] == 0) & dilated_sub_vol
        dilated_volume[slc][mask] = label_id
        
    return dilated_volume

def erode_labeled_volume_3d(labeled_array, iterations=1):
    print(f"Starting 3D erosion ({iterations} iterations)...")
    
    # Create an empty output array of the same shape and type
    eroded_volume = np.zeros_like(labeled_array)
    
    # Define a 3D connectivity structure (the "brush")
    # This creates a cross-shaped 3D kernel for 6-connectivity erosion
    struct = ndimage.generate_binary_structure(3, 1) 
    
    # find_objects returns a list of slices (bounding boxes) for every ID
    # This is the secret to making 3D loops fast
    object_slices = ndimage.find_objects(labeled_array)
    
    for i, slc in enumerate(tqdm(object_slices, desc="Eroding objects")):
        if slc is None:
            continue
            
        label_id = i + 1
        
        # 1. Extract the small sub-volume containing only this object
        sub_vol = labeled_array[slc] == label_id
        
        # 2. Perform 3D erosion on the sub-volume
        eroded_sub_vol = ndimage.binary_erosion(sub_vol, structure=struct, iterations=iterations)
        
        # 3. Place the eroded object back into the global volume
        eroded_volume[slc][eroded_sub_vol] = label_id
    return eroded_volume

def whole_process(input_path, iterations=1, filter_th=1000):
    volume = tifffile.imread(input_path)

    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = volume > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    print("Labeling individual elements...")
    labeled_array = label(binary_mask, connectivity=None)

    # ERODE VOLUMES
    eroded_volume = erode_labeled_volume_3d(labeled_array, iterations=1)
    
    # SPLITING BY INDIVIDUAL OBJECT 
    splited_labels = split_by_object_np(eroded_volume)

    # REMOVE SMALL OBJECTS
    filtered_volume = remove_bellow_th(splited_labels, th=filter_th, output_path=None)

    # DO DILLATION PER CASE 
    dilated_volume = dilate_labeled_volume_3d(filtered_volume, iterations=10)

    # ERODE BACK 
    new_eroded_volume = erode_labeled_volume_3d(dilated_volume, iterations=3)

    # SPLITING BY INDIVIDUAL OBJECT 
    unique_ids = np.unique(new_eroded_volume)
    num_elements = len(unique_ids) - (1 if 0 in unique_ids else 0)
    print(f"Total individual segments found: {num_elements}")

    return new_eroded_volume

eroded_volume = whole_process(
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/2290837_th06.tif", 
    iterations=1, 
    filter_th=1000
    )

# --- Save ---
output_path = '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/filled_2290837_th06.tif'
print(f"Saving to {output_path}...")
tifffile.imwrite(output_path, eroded_volume.astype(np.uint16))

# Save NIfTI (using your preferred orientation)
nii_path = output_path.replace('.tif', '.nii.gz')
nii_data = np.transpose(eroded_volume, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), nii_path)

print("Done.")

Labeling individual elements...
Starting 3D erosion (1 iterations)...


Eroding objects: 100%|██████████| 79/79 [00:00<00:00, 120.36it/s]


Labeling individual elements...
Found 273 individual elements.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [52]:
import numpy as np
import tifffile
import nibabel as nib
from scipy.ndimage import binary_closing
from skimage.morphology import ball # Creates a 3D spherical structure
from tqdm import tqdm 

def fill_cracks_and_holes(labeled_volume, radius=2):
    print(f"Filling gaps/cracks in 3D sheets (Closing Radius: {radius})...")
    
    # Create the result volume
    filled_volume = np.zeros_like(labeled_volume)
    
    # Define the structure for closing (a 3D ball of the given radius)
    # This determines how big a gap can be bridged.
    structure = ball(radius)
    
    unique_labels = np.unique(labeled_volume)
    unique_labels = unique_labels[unique_labels != 0]
    
    modified_ids = []
    
    for label_id in tqdm(unique_labels):
        # 1. Isolate the object
        binary_mask = (labeled_volume == label_id)
        
        # 2. Apply Morphological Closing
        # This connects parts that are close together, effectively filling cracks
        closed_mask = binary_closing(binary_mask, structure=structure)
        
        # 3. Check if changes occurred
        if np.sum(closed_mask) > np.sum(binary_mask):
            modified_ids.append(label_id)
        
        # 4. Store result
        filled_volume[closed_mask] = label_id

    # --- Reporting ---
    print(f"Total objects processed: {len(unique_labels)}")
    print(f"Number of objects modified: {len(modified_ids)}")
    print(f"IDs that were changed: {modified_ids}")

    # --- Saving ---
    print("Saving TIFF...")
    output_tif = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/filtered_closed_2290837.tif"
    tifffile.imwrite(output_tif, filled_volume.astype(np.uint16))
    
    print("Saving NIfTI...")
    output_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/DA/filtered_closed_2290837.nii.gz"
    
    # Apply your specific orientation fixes
    nii_data = np.transpose(filled_volume, (2, 1, 0)).astype(np.uint16)
    nii_data = np.flip(nii_data, axis=0) 
    nii_data = np.flip(nii_data, axis=1)
    
    nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_nii)
    print("Done.")
        
    return filled_volume

# --- Run it ---
# Try radius=2 first. If holes remain, increase to 3 or 4.
filled_volume = fill_cracks_and_holes(
    labeled_volume=final_volume_th,
    radius=4
)

Filling gaps/cracks in 3D sheets (Closing Radius: 4)...


100%|██████████| 13/13 [00:54<00:00,  4.16s/it]


Total objects processed: 13
Number of objects modified: 1
IDs that were changed: [np.uint8(8)]
Saving TIFF...
Saving NIfTI...
Done.


In [48]:
def check_for_holes(labeled_volume):
    print("Verifying internal holes...")
    hole_report = {}
    
    unique_labels = np.unique(labeled_volume)
    unique_labels = unique_labels[unique_labels != 0]
    
    for label_id in unique_labels:
        binary_mask = (labeled_volume == label_id)
        
        # Count original voxels
        original_count = np.count_nonzero(binary_mask)
        
        # Fill and count again
        filled_count = np.count_nonzero(binary_fill_holes(binary_mask))
        
        # If the count increased, there was a hole
        has_hole = filled_count > original_count
        hole_report[label_id] = has_hole
        
        if has_hole:
            print(f"Object {label_id}: Found internal holes ({filled_count - original_count} voxels filled).")
            
    if not any(hole_report.values()):
        print("Verification complete: No internal holes found in any object.")
        
    return hole_report

hole_report = check_for_holes(filled_volume)
hole_report

Verifying internal holes...
Verification complete: No internal holes found in any object.


{np.uint8(1): np.False_,
 np.uint8(2): np.False_,
 np.uint8(3): np.False_,
 np.uint8(4): np.False_,
 np.uint8(5): np.False_,
 np.uint8(7): np.False_,
 np.uint8(8): np.False_,
 np.uint8(11): np.False_,
 np.uint8(12): np.False_,
 np.uint8(13): np.False_,
 np.uint8(16): np.False_,
 np.uint8(32): np.False_,
 np.uint8(42): np.False_}

# Check if it creates bridges

In [1]:
def split_by_object_np(binary_mask):
    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = binary_mask > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    print("Labeling individual elements...")
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    print(f"Found {num_features} individual elements.")
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)
    return final_volume

In [ ]:
import os
from os import listdir
from os.path import join
import tifffile
from skimage.measure import label
import numpy as np
import nibabel as nib

root_pred = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_150/TTA/overlap_0.5/th_0.5"
save_post_pred = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.5"
os.makedirs(save_post_pred, exist_ok=True)
for input_file in listdir(root_pred):
    input_path = join(root_pred, input_file)
    output_path = join(save_post_pred, input_file)

    binary_mask = tifffile.imread(input_path)
    labeled_volume = split_by_object_np(binary_mask) 
    #tifffile.imwrite(f"{output_path}", labeled_volume.astype(np.uint16))
    ni_img = nib.Nifti1Image(labeled_volume.astype(np.uint16), affine=np.eye(4))
    nib.save(ni_img, f"{output_path.replace('.tif', '.nii.gz')}")

    print(output_path)
    

Labeling individual elements...
Found 44 individual elements.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.7/4024884955.tif
